In [1]:
import pandas as pd
import numpy as np
import re
import unidecode
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

basepath = '/home/pcsor/facu/TP2/'
av_detalle = pd.read_csv(basepath + '/entrega6/fiuba_6_avisos_detalle.csv',low_memory=False)
av_navent = pd.read_csv(basepath + '/datos_navent_fiuba/fiuba_6_avisos_detalle.csv',low_memory=False)
av_test = pd.read_csv(basepath + '/test/fiuba_6_avisos_detalle.csv',low_memory=False)
av_missing = pd.read_csv(basepath + '/fiuba_6_avisos_detalle_missing_nivel_laboral.csv',low_memory=False)

In [2]:
av_detalle = pd.concat([av_detalle,av_navent,av_test,av_missing])
av_detalle = av_detalle.drop_duplicates('idaviso')
av_detalle.shape

(25288, 11)

In [3]:
descrip = av_detalle[['idaviso','descripcion','titulo']]

descrip.loc[:,'descripcion'] = descrip['descripcion'].apply(lambda s: re.sub('<.*?>', '', s)) #####saco todo lo que sea codigo HTML
descrip.loc[:,'descripcion'] = descrip['descripcion'].apply(lambda s: re.sub(r'[^\w]', ' ', s)) ##### saco todo lo que no sea alfanumerico
descrip.loc[:,'descripcion'] = descrip['descripcion'].apply(lambda s: s.lower()) ###### lo paso a minuscula
descrip.loc[:,'descripcion'] = descrip['descripcion'].apply(lambda s: unidecode.unidecode(s)) ####### saco los acentos
descrip.loc[:,'descripcion'] = descrip['descripcion'].apply(lambda s: re.sub(r'\b\w{1,3}\b', '', s)) ###### remuevo las palabras cortas
descrip.loc[:,'descripcion'] = descrip['descripcion'].apply(lambda s: re.sub(' +',' ',s)) ###### remuevo los espacios en blanco de mas
descrip.loc[:,'titulo'] = descrip['titulo'].apply(lambda s: s.lstrip()) ###### remuevo los espacios en blanco de mas
descrip.loc[:,'titulo'] = descrip['titulo'].apply(lambda s: re.sub('<.*?>', '', s)) #####saco todo lo que sea codigo HTML
descrip.loc[:,'titulo'] = descrip['titulo'].apply(lambda s: re.sub(r'[^\w]', ' ', s)) ##### saco todo lo que no sea alfanumerico
descrip.loc[:,'titulo'] = descrip['titulo'].apply(lambda s: s.lower()) ###### lo paso a minuscula
descrip.loc[:,'titulo'] = descrip['titulo'].apply(lambda s: unidecode.unidecode(s)) ####### saco los acentos
descrip.loc[:,'titulo'] = descrip['titulo'].apply(lambda s: re.sub(r'\b\w{1,3}\b', '', s)) ###### remuevo las palabras cortas
descrip.loc[:,'titulo'] = descrip['titulo'].apply(lambda s: re.sub(' +',' ',s)) ###### remuevo los espacios en blanco de mas
descrip.loc[:,'titulo'] = descrip['titulo'].apply(lambda s: s.lstrip()) ###### remuevo los espacios en blanco de mas

/home/pcsor/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [4]:
stopw = 'un,de,https,una,unas,unos,uno,sobre,todo,por,tambien,tras,otro,algun,alguno,alguna,algunos,algunas,ser,es,soy,eres,somos,sos,estoy,esta,estamos,estas,estan,como,en\
,para,atras,porque,por que,estado,estaba,ante,antes,siendo,ambos,pero,por,poder,puede,puedo,podemos,podes,pueden,fui,fue,fuimos,fueron,hacer,hago,hace,hacemos,haceis,hacen\
,cada,fin,incluso,primero,desde,ir,voy,va,vamos,vais,van,vaya,bueno,ha,tener,tengo,tiene,tenemos,teneis,tienen,el,la,lo,las,los,su,aqui,mio,tuyo,ellos,ellas,nos,nosotros\
,vosotros,vosotras,si,dentro,solo,solamente,ultimo,largo,haces,usted,ustedes,muchos,aquellos,aquellas,sus,entonces,cierto,ciertos,cierta,ciertas,intentar,intento,intenta,intentas\
,intentais,intentan,dos,bajo,arriba,encima,usar,uso,usas,usa,usamos,usais,usan,emplear,empleo,empleas,emplean,ampleamos,empleais,valor,muy,eras,eramos,eran,modo,bien,cual\
,cuando,donde,mientras,quien,con,entre,sin,podria,podrias,podriamos,podrian,podriais,yo,aquel,intentamos,ev,busqueda,ingresar,queres,haciendo,importante,encuentra\
,encontramos,debe,argentina,nuestro,ubicada,inscribirse,se,in,est,anos,NaN'
stopw = stopw.split(',')

tfidf = TfidfVectorizer(stop_words = stopw,max_features=800)
sparseMT = tfidf.fit_transform(descrip['titulo'])

true_k = 50
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=400, n_init=1)
aver = model.fit_transform(sparseMT)

In [5]:
aver.shape

(25288, 50)

In [7]:
print("10 primeros de cada cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidf.get_feature_names()
for i in range(40):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :20]:
        print(' %s' % terms[ind]),
    print

10 primeros de cada cluster:
Cluster 0:
 eventual
 administrativo
 analista
 cajero
 personal
 meses
 caba
 zona
 bancario
 posicion
 asistente
 entry
 data
 negocios
 impuestos
 operario
 banco
 oficial
 descarga
 varela
Cluster 1:
 gerente
 comercial
 sucursal
 zonal
 ventas
 operaciones
 general
 administracion
 musimundo
 empresa
 finanzas
 busca
 marketing
 produccion
 humanos
 recursos
 zona
 cordoba
 mantenimiento
 planta
Cluster 2:
 quimico
 tecnico
 ingeniero
 calidad
 experiencia
 laboratorio
 zona
 control
 rubro
 comercial
 analista
 textil
 quilmes
 industrial
 alimentos
 martin
 farmaceutico
 operario
 procesos
 mecanico
Cluster 3:
 zona
 norte
 oeste
 caba
 urgente
 tecnico
 belgrano
 analista
 desarrolladores
 operarios
 pilar
 capital
 cajeros
 federal
 empresa
 repositores
 administrativa
 mantenimiento
 asesores
 palermo
Cluster 4:
 produccion
 operario
 operarios
 zona
 asistente
 operarias
 textil
 pilar
 analista
 tecnico
 planificacion
 laboratorio
 linea
 progra

In [11]:
titulos = pd.DataFrame(aver)
#textos = pd.concat([conceptos,titulos],axis=1,ignore_index=True)
textos=titulos.add_prefix('kmeans')
av_detalle.reset_index(drop=True, inplace=True)
av_detalle = pd.concat([av_detalle['idaviso'],textos],axis = 1)
av_detalle.to_csv(basepath + '/tituloskmeans.csv',index=False)

In [12]:
av_detalle

,idaviso,kmeans0,kmeans1,kmeans2,kmeans3,kmeans4,kmeans5,kmeans6,kmeans7,kmeans8,...,kmeans40,kmeans41,kmeans42,kmeans43,kmeans44,kmeans45,kmeans46,kmeans47,kmeans48,kmeans49
0,1111556097,1.260392,1.170406,1.215642,1.235588,1.113129,1.055710,0.993548,1.215690,1.144794,...,1.210203,1.141802,1.174872,1.248274,1.159300,1.194624,1.215217,1.260614,1.218426,1.184977
1,1111949392,1.259310,1.167272,1.213726,1.233387,1.127160,1.053335,1.001999,1.215690,1.140248,...,1.220749,1.145989,1.180691,1.263882,1.153639,1.194780,1.214721,1.264187,1.194339,1.182895
2,1112145935,1.257266,1.174936,1.215642,1.235588,1.134334,1.056967,0.998672,1.215690,1.145801,...,1.211430,1.150798,1.182806,1.263882,1.160470,1.198275,1.215217,1.258859,1.224837,1.190474
3,1112146010,1.261110,1.174936,1.215642,1.102925,1.134334,1.060022,1.001920,1.215690,1.145801,...,1.220749,1.151961,1.182806,1.263882,1.161227,1.198275,1.215217,1.264187,1.220535,1.190474
4,1112211475,1.259395,1.173649,1.215642,1.235588,1.133922,1.057621,0.990805,1.215690,1.145801,...,1.220749,1.149738,1.182806,1.263882,1.159945,1.198275,1.213933,1.264187,1.224837,1.190474
5,1112211493,1.239418,1.169022,1.214316,1.235588,1.123560,1.040461,0.979973,1.215690,1.126424,...,1.220749,1.149458,1.171133,1.263882,1.156717,1.198275,1.214814,1.264187,1.224837,1.169962
6,1112211531,1.259498,1.158482,1.215642,1.233427,1.118417,1.051412,0.997709,1.199691,1.137285,...,0.893909,1.149272,1.170979,1.263882,1.147786,1.193851,1.211924,1.227497,1.201705,1.188388
7,1112211553,1.256477,1.174080,1.147198,1.235588,1.131989,1.054009,0.998951,1.205756,1.145801,...,1.219046,1.151961,1.182806,1.260430,1.145237,1.195532,1.215217,1.261073,1.224837,1.180614
8,1112211555,1.162948,1.166513,1.215642,1.195930,1.109005,1.048278,0.997870,1.215690,1.144397,...,1.178028,1.150785,1.178453,1.263882,1.156487,1.198275,1.209937,1.143101,1.168689,1.190474
9,1112211559,1.261110,1.174306,1.194648,1.235588,1.134042,1.057833,0.998960,1.215690,1.145084,...,1.220749,1.151342,1.180278,1.263882,1.145909,1.198275,1.215217,1.264187,1.221520,1.153882
